    датафрема log нету в материалах занятия. есть только visit_log.csv я использовал его.
 

In [1]:
import pandas as pd


In [2]:
log_file = "3_home_work/data/visit_log.csv"
df = pd.read_csv(log_file, sep=';')
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


# Задание 1


In [3]:
def source_type(row):
    if row['traffic_source'] in ['yandex', 'google']:
        return 'organic'
    elif row['traffic_source'] in ['paid', 'email']:
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return row['traffic_source']

df['source_type'] = df.apply(source_type, axis=1)

In [4]:
# df.head(10)

# Задание 2

In [5]:
file_url = "3_home_work/data/URLs.txt" 

In [6]:
with open(file_url, 'r') as f:
    urls = f.readlines()


In [7]:
urls = pd.DataFrame(urls, columns=['url'])
urls.head()

,url
0,url\n
1,/world/\n
2,/latest/\n
3,/?updated=top\n
4,/politics/36188461-s-marta-zhizn-rossiyan-susc...


In [8]:

filtred_url = urls[urls.url.str.contains(r'/\d{8}-', regex=True)].copy()

In [9]:
filtred_url.head()

,url
4,/politics/36188461-s-marta-zhizn-rossiyan-susc...
5,/world/36007585-tramp-pridumal-kak-reshit-ukra...
6,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
7,/video/36001498-poyavilis-pervye-podrobnosti-g...
8,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

In [32]:
rating = pd.read_csv('3_home_work/data/ml-latest-small/ratings.csv')
rating

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [33]:
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')


In [34]:
rating['userId'].value_counts()

userId
547    2391
564    1868
624    1735
15     1700
73     1610
       ... 
296      20
289      20
249      20
221      20
1        20
Name: count, Length: 671, dtype: int64

In [40]:
# Calculate the number of ratings per user
user_rating_counts = rating['userId'].value_counts()

# Filter users who have more than 100 ratings
users_with_many_ratings = user_rating_counts[user_rating_counts > 100].index

# Create a new DataFrame with ratings from users who have more than 100 ratings
rating100 = rating[rating['userId'].isin(users_with_many_ratings)].copy()



In [41]:
rating100

,userId,movieId,rating,timestamp
147,4,10,4.0,2000-02-06 04:17:25
148,4,34,5.0,2000-02-07 10:32:36
149,4,112,5.0,2000-02-06 04:16:22
150,4,141,5.0,2000-02-07 10:34:41
151,4,153,4.0,2000-02-06 04:29:06
...,...,...,...,...
99999,671,6268,2.5,2003-10-08 02:16:10
100000,671,6269,4.0,2003-10-03 02:46:41
100001,671,6365,4.0,2003-12-09 03:26:03
100002,671,6385,2.5,2003-12-09 14:21:03


In [42]:
# посчитать время жизни пользователя по каждому пользователю
rating100['timestamp'] = pd.to_datetime(rating100['timestamp'], unit='s')
rating100['date'] = rating100['timestamp'].dt.date # выделим дату
rating100['date'] = pd.to_datetime(rating100['date']) # приведем к формату datetime


In [50]:
t = rating100.groupby('userId').agg({'date': ['min', 'max']}).reset_index()

In [51]:
t.columns

MultiIndex([('userId',    ''),
            (  'date', 'min'),
            (  'date', 'max')],
           )

In [52]:
t.columns = ['userId', 'date_min', 'date_max']

In [54]:
t['lifetime'] = t['date_max'] - t['date_min']
t

,userId,date_min,date_max,lifetime
0,4,2000-02-05,2000-02-08,3 days
1,8,2006-07-31,2006-08-01,1 days
2,15,2001-08-16,2016-07-24,5456 days
3,17,2005-09-23,2005-09-23,0 days
4,19,1997-02-06,1997-02-06,0 days
...,...,...,...,...
253,656,2001-04-02,2001-04-02,0 days
254,659,1996-06-12,1997-06-13,366 days
255,664,2012-07-31,2015-09-10,1136 days
256,665,2001-06-18,2003-03-06,626 days


In [55]:
### К домашнему заданию №4
# Дана статистика услуг перевозок клиентов компании по типам:
# - rzd - железнодорожные перевозки
# - auto - автомобильные перевозки
# - air - воздушные перевозки
# - client_base - адреса клиентов
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [56]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента
revenue = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
revenue


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [57]:
# Аналогично предыдущему, но с указанием адреса клиента
revenue_full = revenue.merge(client_base, how='outer', on='client_id')
revenue_full

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
